## THE CIRCUIT CONFIGURATION

In [1]:
import subprocess
import sys
#subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
#subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])


import json
import ezkl
import onnx
!pip list | grep ezkl
!pip list | grep solc-select
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image

import logging
import os
import sys
import time
from datetime import datetime


# uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

: 

### CAM circuit calibration

In [9]:
model_path = os.path.join('outputs/cam/modelGetcam460.onnx')
settings_path = os.path.join('outputs/cam/settings.json')
cal_path = os.path.join('outputs/cam/calib_dataGetcam460.json')

#For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
#This cell instantiates some parameters that determine the circuit shape, size etc
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

os.environ["RUST_BACKTRACE"] = "1"

!RUST_LOG=trace
# TODO: Dictionary outputs
start_time = time.time()
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
end_time = time.time()
assert res == True
print("gen_settings done in: ", end_time - start_time, "seconds")

os.environ["RUST_BACKTRACE"] = "1"

# *** SCALES is one of main knobs you can turn to trade off accuracy for proving efficiency
# Under the hood calibration iterates over the scales array to see how precise you can go before failure.
# For example if 7 fails it falls back to 1.
# (Right now the default scales values for the resources target is 8-10 and 10-13  for accuracy)
start_time = time.time()
#res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "accuracy")
res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", max_logrows=20)#, scales=[6])
end_time = time.time()
print("calibrate_settings done in", end_time - start_time, "seconds")

gen_settings done in:  0.042871952056884766 seconds


INFO ezkl.execute 2024-07-08 09:58:02,685 execute.rs:1037 num calibration batches: 20
WARNING ezkl.execute 2024-07-08 09:58:53,466 execute.rs:1356 

 <------------- Numerical Fidelity Report (input_scale: 12, param_scale: 12, scale_input_multiplier: 1) ------------->

+-----------------+--------------+--------------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error      | median_error | max_error    | min_error      | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-----------------+--------------+--------------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.0000033193216 | 0            | 0.0006392002 | -0.00047534704 | 0.00004725402  | 0                | 0.0006392002  | 0     

calibrate_settings done in 51.54940891265869 seconds


In [10]:
# Visualize the circuit settings
json_file_path = settings_path
# Read the content of the JSON file
with open(json_file_path, 'r') as file:
    json_content = json.load(file)

# Print the JSON content
print(json.dumps(json_content, indent=2))

{
  "run_args": {
    "tolerance": {
      "val": 0.0,
      "scale": 1.0
    },
    "input_scale": 12,
    "param_scale": 12,
    "scale_rebase_multiplier": 1,
    "lookup_range": [
      -34192,
      28276
    ],
    "logrows": 16,
    "num_inner_cols": 2,
    "variables": [
      [
        "batch_size",
        1
      ]
    ],
    "input_visibility": "Public",
    "output_visibility": "Public",
    "param_visibility": "Fixed",
    "div_rebasing": false,
    "rebase_frac_zero_constants": false,
    "check_mode": "UNSAFE",
    "commitment": "KZG"
  },
  "num_rows": 50066,
  "total_assignments": 100109,
  "total_const_size": 4,
  "total_dynamic_col_size": 0,
  "num_dynamic_lookups": 0,
  "num_shuffles": 3,
  "total_shuffle_col_size": 5003,
  "model_instance_shapes": [
    [
      1,
      3
    ],
    [
      1,
      3,
      50,
      50
    ],
    [
      1,
      50,
      50
    ]
  ],
  "model_output_scales": [
    12
  ],
  "model_input_scales": [
    12,
    12
  ],
  "module

### CAM Setup

In [ ]:
# Compile the model into a circuit
compiled_model_path = os.path.join('outputs/cam/getcam460.ezkl')
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

# Get public srs from kzg ceremony.
settings_path = os.path.join('prover/outputs/cam/settings.json')
res = ezkl.get_srs(settings_path)

In [14]:
import os
import time
import psutil

# Setup the circuit and make sure the keys are generated afterwards.
pk_path = os.path.join('outputs/cam/pkGetcam460.key')
vk_path = os.path.join('outputs/cam/vkGetcam460.key')

# Start time and process
start_time = time.time()
process = psutil.Process(os.getpid())

os.environ["RUST_BACKTRACE"] = "1"

# Capture initial CPU and RAM usage
initial_cpu_times = process.cpu_times()
initial_memory_info = process.memory_info()

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

# Capture end CPU and RAM usage
final_cpu_times = process.cpu_times()
final_memory_info = process.memory_info()

end_time = time.time()
execution_time = end_time - start_time

vk_size = os.path.getsize(vk_path)
pk_size = os.path.getsize(pk_path)

# Calculate CPU and RAM usage
cpu_user_time = final_cpu_times.user - initial_cpu_times.user
cpu_system_time = final_cpu_times.system - initial_cpu_times.system
ram_usage_mb = (final_memory_info.rss - initial_memory_info.rss) / (1024 * 1024)  # Convert bytes to megabytes

print("setup done in: ", execution_time, "seconds")
print("CPU user time: ", cpu_user_time, "seconds")
print("CPU system time: ", cpu_system_time, "seconds")
print("RAM usage: ", ram_usage_mb, "MB")

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

INFO ezkl.graph.model 2024-07-08 10:01:21,151 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-08 10:01:23,552 mod.rs:520 VK took 2.406
INFO ezkl.graph.model 2024-07-08 10:01:23,557 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-08 10:01:26,088 mod.rs:526 PK took 2.535
INFO ezkl.pfsys 2024-07-08 10:01:26,092 mod.rs:844 done saving verification key ✅
INFO ezkl.pfsys 2024-07-08 10:01:26,683 mod.rs:827 done saving proving key ✅
ERROR asyncio 2024-07-08 10:01:26,706 base_events.py:1738 Future exception was never retrieved
future: <Future finished exception=RuntimeError('Failed to get srs: [Uncategorized] You will need to provide a valid settings file to use the settings option. You should run gen-settings to generate a settings file (and calibrate-settings to pick optimal logrows).')>
RuntimeError: Failed to get srs: [Uncategorized] You will need to provide a valid settings file to use the settings option. You should run gen-settings to generate a settings file (and calibrate-s

setup done in:  5.570791721343994 seconds
CPU user time:  19.594690560000032 seconds
CPU system time:  1.6920903679999952 seconds
RAM usage:  289.328125 MB


## CAM Proving
### here we will generate and verify a proof locally. Then we will format the inputs and the proof in a way compatible for the evm verifier

In [15]:
witness_path = os.path.join('outputs/cam/witness460.json')
data_path = os.path.join('outputs/cam/inputGetcam460.json')

# generate the witness file
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)

In [26]:
# Generate the proof
import os
import time
import psutil
import json

# Generate the proof
proof_path = os.path.join('outputs/cam/proofGetcamSINGLE460.json')

process = psutil.Process(os.getpid())  # Get the current process
start_time = time.time()
start_cpu_times = process.cpu_times()  # CPU times at the start
start_memory = process.memory_info().rss  # Memory usage at the start

proof = ezkl.prove(
    witness_path,
    compiled_model_path,
    pk_path,
    proof_path,
    "single",  # To produce an aggregated EVM proof using poseidon for the smaller proof
)

end_time = time.time()
end_cpu_times = process.cpu_times()  # CPU times at the end
end_memory = process.memory_info().rss  # Memory usage at the end

execution_time = end_time - start_time
cpu_time_user = end_cpu_times.user - start_cpu_times.user
cpu_time_system = end_cpu_times.system - start_cpu_times.system
total_cpu_time = cpu_time_user + cpu_time_system
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

print(f"Proof generation took {execution_time:.2f} seconds (real-time).")
print(f"CPU time used: User - {cpu_time_user:.2f} seconds, System - {cpu_time_system:.2f} seconds.")
print(f"Total CPU time: {total_cpu_time:.2f} seconds.")
print(f"Memory used: {memory_used:.2f} MB")
print(proof)


INFO ezkl.pfsys 2024-07-08 11:02:14,935 mod.rs:810 loaded proving key ✅
INFO ezkl.pfsys 2024-07-08 11:02:14,943 mod.rs:581 proof started...
INFO ezkl.graph.model 2024-07-08 11:02:14,957 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-08 11:02:24,693 mod.rs:620 proof took 9.749


Proof generation took 10.23 seconds (real-time).
CPU time used: User - 38.96 seconds, System - 1.20 seconds.
Total CPU time: 40.16 seconds.
Memory used: 733.47 MB
{'instances': [['...']], 'proof': '0x235512c2849102b771431e082c1ddafa88bcafce13c49bf4fca909fd428d171f2f5edaaf1014f0581fc1f687b3489b8a86983a2ca3f764bcbd8163660ab0f6151c33c6c607c173cf4cbb4e35aeae8862e92ad17b8abd3fa67aeac0c75cd39abb1366457aeb5b1b4c92887900d5a5820349ea02609b1073876c55ee0487c9fe671c45fef349b4b532bf84972e1ba7d46b9516fad091324a894fdad8a33d87907706216f1f8529fc0feccda9a36d5390279f90965757363e2af734cae4d6f84a040c990040e62513e40e93a1c46dfa7342211b80aee67262261563d8d1fc60f553020581d5d9e9230a4c43cda91d7387899175de36012e91891202c1bdda25b41d199e19682f45b8a6dc66fdc720a6764866609cef2a404e2c97ebe939962e5fd205004b98daa8d1e234d74a1af7e2b005b6acf3c0d41110ceb432d369fde922b4064f9fcf33e2d8ae3d2b856ab36d36db46bb3159b252c23b465d7d6ff2c9677d2f5eefe13acc1247dc85f53934b0267af576b5c05e59eb77f7aea1110c1d87250b02b95909e1976b0fb2e955225d1fe7

In [17]:
# Generate a larger SRS. This is needed for the aggregated proof

res = await ezkl.get_srs(settings_path=None, logrows=23, commitment=ezkl.PyCommitments.KZG)

INFO ezkl.execute 2024-07-08 10:15:44,217 execute.rs:741 SRS already exists at that path
INFO ezkl.execute 2024-07-08 10:15:44,535 execute.rs:639 read 1073742084 bytes from file (vector of len = 1073742084)
INFO ezkl.execute 2024-07-08 10:15:47,553 execute.rs:646 file hash: 748e48b9b6d06f9c82d26bf551d0af43ee2e801e4be56d7ccb20312e267fd1d6


In [20]:
import os

def get_json_file_paths(proofs_dir):
    json_file_paths = []
    for root, dirs, files in os.walk(proofs_dir):
        for file in files:
            if file.endswith('.json'):
                json_file_paths.append(os.path.join(root, file))
    return json_file_paths

proofs_dir = 'outputs/cam/aggr/proofs/460'
all_proofs = get_json_file_paths(proofs_dir)
print(all_proofs)

['outputs/cam/aggr/proofs/460/proof_witness_input460_2.json', 'outputs/cam/aggr/proofs/460/proof_witness_input460_0.json', 'outputs/cam/aggr/proofs/460/proof_witness_input460_1.json']


In [21]:
proof_path = os.path.join('outputs/cam/proofGetcam460.json')
all_proofs.append(proof_path)
print(all_proofs)

['outputs/cam/aggr/proofs/460/proof_witness_input460_2.json', 'outputs/cam/aggr/proofs/460/proof_witness_input460_0.json', 'outputs/cam/aggr/proofs/460/proof_witness_input460_1.json', 'outputs/cam/proofGetcam460.json']


In [23]:
# Run mock aggregate to check whether the proof works
# Use mock to check for validity as it takes a shorter time to check compared to a full aggregated proof
res = ezkl.mock_aggregate(all_proofs, 24)
assert res == True

In [30]:
# Sanity check off-chain verification
proof_path = os.path.join('outputs/cam/proofGetcamSINGLE460.json')
vk_path = os.path.join('outputs/cam/vkGetcam460.key')
settings_path = os.path.join('outputs/cam/settings.json')


process = psutil.Process(os.getpid())  # Get the current process
start_time = time.time()
start_cpu_times = process.cpu_times()  # CPU times at the start
start_memory = process.memory_info().rss  # Memory usage at the start

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

eend_time = time.time()
end_cpu_times = process.cpu_times()  # CPU times at the end
end_memory = process.memory_info().rss  # Memory usage at the end

execution_time = end_time - start_time
cpu_time_user = end_cpu_times.user - start_cpu_times.user
cpu_time_system = end_cpu_times.system - start_cpu_times.system
total_cpu_time = cpu_time_user + cpu_time_system
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

print(f"Proof generation took {execution_time:.2f} seconds (real-time).")
print(f"CPU time used: User - {cpu_time_user:.2f} seconds, System - {cpu_time_system:.2f} seconds.")
print(f"Total CPU time: {total_cpu_time:.2f} seconds.")
print(f"Memory used: {memory_used:.2f} MB")

assert res == True
print("verified locally")

INFO ezkl.pfsys 2024-07-08 11:58:29,578 mod.rs:787 loaded verification key ✅
INFO ezkl.execute 2024-07-08 11:58:29,627 execute.rs:2453 verify took 0.48
INFO ezkl.execute 2024-07-08 11:58:29,627 execute.rs:2458 verified: true


Proof generation took -43.32 seconds (real-time).
CPU time used: User - 0.08 seconds, System - 0.01 seconds.
Total CPU time: 0.09 seconds.
Memory used: 5.16 MB
verified locally
